In [1]:
# What did I learned so far:
# 1) you do the best for the mean encoding of items in stock and shops open
#  and train a separate model for those
# 2) for the rest you force zero the discontinued items and closed shops
# 3) for the sparse shops and items you predict the mean of last tree month 

# Prepare data

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# Load data
train_df = pd.read_csv("../sales_train.csv.gz")
test_df  = pd.read_csv("../test.csv.gz")

categories = pd.read_csv('../item_categories.csv')
items = pd.read_csv('../items.csv')

In [3]:
%store -r __trends_combined

In [4]:
%%time
__trends_combined

CPU times: user 1min 34s, sys: 48 s, total: 2min 22s
Wall time: 2min 30s


/usr/local/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:3947: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


[0, 1, 8, 11, 13, 17, 23, 27, 29, 30, 32, 33, 40, 43, 54]
15 out of 60 shops were not selling at least last 3 months
Sanity check: known item categories is 84 and total categories is 84
Total number of items is 22170, among those sold are 21807
Never sold are 363 items from 39 categories of 12 types and 32 subtypes
 (in total there are 20 types and 65 subtypes)
21807


/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


No sales for 19 categories, 4 types, and 15 subtypes
14627 out of 21807 items were not sold at least last 3 months
{27}
{16689, 15106, 20539, 5613}
CPU times: user 2min 21s, sys: 1min 15s, total: 3min 36s
Wall time: 3min 50s


# Construct some trands, finally

In [23]:
group = train_df.groupby(['item_id']).agg({'item_price': 'mean'})
group.columns = ['item_avg_item_price']
group.reset_index(inplace=True)

all_data = pd.merge(all_data, group, on=['item_id'], how='left')
matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)

group = train.groupby(['date_block_num','item_id']).agg({'item_price': ['mean']})
group.columns = ['date_item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

lags = [1,2,3,4,5,6]
matrix = lag_feature(matrix, lags, 'date_item_avg_item_price')

for i in lags:
    matrix['delta_price_lag_'+str(i)] = \
        (matrix['date_item_avg_item_price_lag_'+str(i)] - matrix['item_avg_item_price']) / matrix['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if row['delta_price_lag_'+str(i)]:
            return row['delta_price_lag_'+str(i)]
    return 0
    
matrix['delta_price_lag'] = matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)
matrix['delta_price_lag'].fillna(0, inplace=True)

# https://stackoverflow.com/questions/31828240/first-non-null-value-per-row-from-a-list-of-pandas-columns/31828559
# matrix['price_trend'] = matrix[['delta_price_lag_1','delta_price_lag_2','delta_price_lag_3']].bfill(axis=1).iloc[:, 0]
# Invalid dtype for backfill_2d [float16]

fetures_to_drop = ['item_avg_item_price', 'date_item_avg_item_price']
for i in lags:
    fetures_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    fetures_to_drop += ['delta_price_lag_'+str(i)]

matrix.drop(fetures_to_drop, axis=1, inplace=True)


,date_block_num,shop_id,item_id,total_monthly_shop_sales_lag_1,trg_shop_lag1
8117,0,1,28,NaN,0.00000
373345,1,1,28,0.361306,1.91987
